<a href="https://colab.research.google.com/github/Jdmolano2020/TesisNer/blob/main/CargueDatasetSROIE_4TextAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00


##Cargar dataset Sroie

In [ ]:
# limpiamos rastros del dataset y descomprimimos
!rm -rf /tmp/SROIE2019v02
!unzip /content/drive/MyDrive/archive_SROIEv2.zip -d /tmp/SROIE2019v02/

Archive:  /content/drive/MyDrive/archive_SROIEv2.zip
   creating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/config.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/pytorch_model.bin  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/special_tokens_map.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/tokenizer_config.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/training_args.bin  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/vocab.txt  
   creating: /tmp/SROIE2019v02/SROIE2019/test/
   creating: /tmp/SROIE2019v02/SROIE2019/test/box/
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X00016469670.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X00016469671.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X51005200931.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X51005230605.txt  
  inflating: /t

##Importar Librerias

In [ ]:
import pandas as pd
import os
import json
import random
import re
import joblib
from tqdm import tqdm

import spacy.cli
from spacy import displacy
from spacy.util import minibatch, compounding
from spacy.training import Example
from spacy.scorer import Scorer
import pathlib

In [ ]:
from spacy.training import offsets_to_biluo_tags
from spacy.tokenizer import Tokenizer

In [ ]:
# Instale la versión más reciente de gensim.
# De lo contrario, puede recibir el siguiente error al ejecutar naw.WordEmbsAug():
# El objeto 'Word2VecKeyedVectors' no tiene atributo 'index_to_key'
# see: https://stackoverflow.com/questions/71032760/word2veckeyedvectors-object-has-no-attribute-index-to-key
!pip install --upgrade gensim --quiet

In [ ]:
# Importar gensim.
# Nota: deberá reiniciar el tiempo de ejecución para importar la versión más reciente de gensim
import gensim
print(gensim.__version__)

In [ ]:
# Instale el tokenizador que necesita el modelo de traducción inversa
!pip install sacremoses --quiet

In [ ]:
# Install the tokenizer
import sacremoses

In [ ]:
# Install the nlpaug module
!pip install nlpaug --quiet

In [ ]:
# Importar el módulo nlpaug y sus métodos
import nlpaug.augmenter.word as naw
import nlpaug.flow as nafc
from nlpaug.util import Action

In [ ]:
# Install Huggingface Transformers
! pip install transformers[sentencepiece] --quiet

In [ ]:
# Install datasets
! pip install datasets --quiet

In [ ]:
! pip install accelerate -U

In [ ]:
# Import other modules
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import numpy as np
import matplotlib.pyplot as plt
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import gc

## Crear Modelo NER

In [ ]:
nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner")

In [ ]:
tokenizer = nlp.tokenizer

## Crear Funciones

In [ ]:
def parse(line):
    fields = line.strip().split(",")
    if len(fields) == 9:
        return fields
    else:
        return fields[:8] + [",".join(fields[8:])]

def build_text(data):
    text = " ".join(data.text)
    text = text.replace("  "," ")
    return text

def entidades_total (ruta,archivo):
  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)

  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  #print("entities=",entities)
  #print("type_entities=",type(entities))
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      ent_text=ent_text.translate(str.maketrans({"-":  r"\-",
                                                  "]":  r"\]",
                                                  "\\": r"\\",
                                                  "^":  r"\^",
                                                  "$":  r"\$",
                                                  "*":  r"\*",
                                                  ".":  r"\.",
                                                  "(":  r"\(",
                                                  ")":  r"\)" }))
    #print("ent_text=",ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()

  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  #print("entidades",entidades)
  entidades_n = []
  for ent in reversed(entidades):
    #print("ent=",ent)
    label= ent[2]
    if label_iden[label]==False:
      label_iden[label]=True
      entidades_n.append(ent)
    #print("entidades_n=",entidades_n)
  entidades_n.sort()
  #print("entidades_n",entidades_n)
  texto_faltantes=[]
  i=0
  pos_ini = 0
  pos_fin = 0
  #print("entidades_n=",entidades_n)
  for ent in entidades_n:
    if (i == 0) & (ent[0] > 0):
      pos_ini = 0
      pos_fin = ent[0]-1
      texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
      #print("ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    if i < len(entidades_n)-1:
      if ent[1]+1 != entidades_n[i+1][0]:
        pos_ini = entidades_n[i][1]+1
        pos_fin = entidades_n[i+1][0]-1
        texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
        #print("ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    i+=1

  if (pos_fin < len(texto)-1) & (entidades_n[-1][1] < len(texto)-1):
    if pos_fin > entidades_n[-1][1]:
      pos_ini = pos_fin + 1
      pos_fin = len(texto)
    else:
      pos_ini = entidades_n[-1][1]+1
      pos_fin = len(texto)
    texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
    #print("pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
  entidades_faltantes = []
  #print("texto_faltantes",texto_faltantes)
  for texto_fa in texto_faltantes:
    pos_ini,pos_fin,texto_f = texto_fa
    #print("pos_ini=",pos_ini," pos_fin=",pos_fin," texto_f=",texto_f)
    #tonkens = tokenizer(texto_f)
    tonkens = texto_f.split(' ')
    pos_ini_t = pos_ini
    for token in tonkens:
      pos_fin_t = pos_ini_t + len(token)
      #print("pos_ini_t=",pos_ini_t,"pos_fin_t=",pos_fin_t,"token=",token,"textoP=",texto[pos_ini_t:pos_fin_t])
      entidades_faltantes.append((pos_ini_t,pos_fin_t,'O'))
      pos_ini_t = pos_fin_t + 1
  #print("entidades_faltantes",entidades_faltantes)
  entidades_final=entidades_n + entidades_faltantes
  entidades_final.sort()
  spacy.training.offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  tags = offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  #print("tags=",tags)
  #doc_p = nlp.make_doc(texto)
  #for tok in doc_p:
  #  print("tok=",tok)
  entidades_final=dict(entities=entidades_final)
  entidades_final= (texto,entidades_final,entidades_n,archivo)
  print("entidades_final=",entidades_final)
  return(entidades_final)

##Validacion entidades

In [ ]:
def entidades_total_prueba_entidad (ruta,archivo):
  #ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
  #archivo = 'X51007419197.txt'

  #print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)

  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  #print("entities=",entities)
  #print("type_entities=",type(entities))
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      #ent_text=re.escape(ent_text)
      ent_text=ent_text.translate(str.maketrans({"-":  r"\-",
                                                  "]":  r"\]",
                                                  "\\": r"\\",
                                                  "^":  r"\^",
                                                  "$":  r"\$",
                                                  "*":  r"\*",
                                                  ".":  r"\.",
                                                  "(":  r"\(",
                                                  ")":  r"\)" }))
    #print("ent_text=",ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()
  #print("entidades",entidades)
  #print("type_entidades=",type(entidades))
  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  #print("entidades",entidades)
  entidades_n = []
  for ent in reversed(entidades):
    #print("ent=",ent)
    label= ent[2]
    if label_iden[label]==False:
      label_iden[label]=True
      entidades_n.append(ent)
  entidades_n.sort()
  #print("entidades_n=",entidades_n)
  entidadesnoen = []
  for name, ent_text in entities.items():
    #print(name)
    ent_e=0
    for elemeto in entidades_n:
      #print(elemeto[2])
      if name == elemeto[2]:
        ent_e+=1
    if ent_e == 0:
      entidadesnoen.append((name,archivo))
  if len(entidadesnoen)>0:
    print("entidadesnoen",entidadesnoen)
  return(entidadesnoen)

In [ ]:
def entidades_total_prueba (ruta,archivo):
  #ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
  #archivo = 'X51007419197.txt'

  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)

  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  #print("entities=",entities)
  #print("type_entities=",type(entities))
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      ent_text=ent_text.translate(str.maketrans({"-":  r"\-",
                                                  "]":  r"\]",
                                                  "\\": r"\\",
                                                  "^":  r"\^",
                                                  "$":  r"\$",
                                                  "*":  r"\*",
                                                  ".":  r"\.",
                                                  "(":  r"\(",
                                                  ")":  r"\)" }))
    #print("ent_text=",ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()

  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  #print("entidades",entidades)
  entidades_n = []
  for ent in reversed(entidades):
    #print("ent=",ent)
    label= ent[2]
    if label_iden[label]==False:
      label_iden[label]=True
      entidades_n.append(ent)
    #print("entidades_n=",entidades_n)
  entidades_n.sort()
  #print("entidades_n",entidades_n)
  texto_faltantes=[]
  i=0
  pos_ini = 0
  pos_fin = 0

  for ent in entidades_n:
    if (i == 0) & (ent[0] > 0):
      pos_ini = 0
      pos_fin = ent[0]-1
      texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
      #print("if 1 ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    if i < len(entidades_n)-1:
      if ent[1]+1 != entidades_n[i+1][0]:
        pos_ini = entidades_n[i][1]+1
        pos_fin = entidades_n[i+1][0]-1
        texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
        #print("if 2 ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    i+=1

  if (pos_fin < len(texto)-1) & (entidades_n[-1][1] < len(texto)-1):
    if pos_fin > entidades_n[-1][1]:
      pos_ini = pos_fin + 1
      pos_fin = len(texto)
    else:
      pos_ini = entidades_n[-1][1]+1
      pos_fin = len(texto)
    texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
    #print("if 3 ent=",ent, "pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
  entidades_faltantes = []
  #print("texto_faltantes",texto_faltantes)
  for texto_fa in texto_faltantes:
    pos_ini,pos_fin,texto_f = texto_fa
    #print("pos_ini=",pos_ini," pos_fin=",pos_fin," texto_f=",texto_f)
    #tonkens = tokenizer(texto_f)
    tonkens = texto_f.split(' ')
    pos_ini_t = pos_ini
    for token in tonkens:
      pos_fin_t = pos_ini_t + len(token)
      #print("pos_ini_t=",pos_ini_t,"pos_fin_t=",pos_fin_t,"token=",token,"textoP=",texto[pos_ini_t:pos_fin_t])
      entidades_faltantes.append((pos_ini_t,pos_fin_t,'O'))
      pos_ini_t = pos_fin_t + 1
  #print("entidades_faltantes",entidades_faltantes)
  entidades_final=entidades_n + entidades_faltantes
  entidades_final.sort()
  spacy.training.offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  tags = offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  #print("tags=",tags)
  #doc_p = nlp.make_doc(texto)
  #for tok in doc_p:
   #print("tok=",tok)
  entidades_final=dict(entities=entidades_final)
  entidades_final= (texto,entidades_final,entidades_n)
  #print("entidades_final=",entidades_final)
  return(entidades_final)

Nueva funcion cargue

In [ ]:
def entidades_total_opc2 (ruta,archivo):
  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  texto = build_text(data)
  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      ent_text=re.escape(ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()
  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  print("entidades",entidades)
  entidades_n = []
  for ent in entidades:
    label= ent[2]
    if label_iden[label]==False:
      if (label=='date') & (label_iden['total']==True):
        continue
      else:
        label_iden[label]=True
        entidades_n.append(ent)
  print("entidades_n",entidades_n)
  texto_faltantes=[]
  entidades_faltantes = []
  i=0
  pos_ini = 0
  pos_fin = 0
  for ent in entidades_n:
    if (i == 0) & (ent[0] > 0):
      pos_ini = 0
      pos_fin = ent[0]-1
      texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
      entidades_faltantes.append((pos_ini,pos_fin,'O'))
    else:
      if i < len(entidades_n)-1:
        if ent[1]+1 != entidades_n[i+1][0]:
          pos_ini = entidades_n[i][1]+1
          pos_fin = entidades_n[i+1][0]-1
          texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
          entidades_faltantes.append((pos_ini,pos_fin,'O'))
    i+=1
  if (pos_fin < len(texto)-1) & (entidades_n[-1][1] < len(texto)-1):
    if pos_fin > entidades_n[-1][1]:
      pos_ini = pos_fin + 1
      pos_fin = len(texto)
    else:
      pos_ini = entidades_n[-1][1]+1
      pos_fin = len(texto)
    texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
    entidades_faltantes.append((pos_ini,pos_fin,'O'))

  print("texto_faltantes",texto_faltantes)
  print("entidades_faltantes",entidades_faltantes)
  entidades_final=entidades_n + entidades_faltantes
  entidades_final.sort()
  entidades_final=dict(entities=entidades_final)
  entidades_final= (texto,entidades_final,archivo)
  return(entidades_final)

##Armar Dataset

In [ ]:
data_dir = "/tmp/SROIE2019v02/SROIE2019/train/box"
data_dir = pathlib.Path(data_dir)
box_count = len(list(data_dir.glob('**/*.txt')))
print(box_count)
l_box=list(data_dir.glob('**/*.txt'))

data_dir = "/tmp/SROIE2019v02/SROIE2019/test/box"
data_dir = pathlib.Path(data_dir)
box_count_test = len(list(data_dir.glob('**/*.txt')))
print(box_count_test)
l_box_test=list(data_dir.glob('**/*.txt'))

626
347


In [ ]:
l_box_archivo= list(map(lambda cadena: str(cadena)[-16:], l_box))

In [ ]:
l_box_test_archivo= list(map(lambda cadena: str(cadena)[-16:], l_box_test))

Revisar entidades train

In [ ]:
resul= []
ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
#archivo = 'X51006913060.txt'
#resul=entidades_total_prueba_entidad (ruta,archivo)
for archivo in l_box_archivo:
  resul=entidades_total_prueba_entidad (ruta,archivo)


Revisar entidades test

In [ ]:
resul= []
ruta ='/tmp/SROIE2019v02/SROIE2019/test/'
#archivo = 'X51006619842.txt'
#resul=entidades_total_prueba_entidad (ruta,archivo)
for archivo in l_box_test_archivo:
  resul=entidades_total_prueba_entidad (ruta,archivo)

Revisar spacy train

In [ ]:
resul= []
ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
#archivo = 'X51005719904.txt'
#resul=entidades_total_prueba (ruta,archivo)
for archivo in l_box_archivo:
 resul=entidades_total_prueba (ruta,archivo)

X51005447850.txt
X51006620189.txt
X51007103641.txt
X51006557179.txt
X00016469612.txt
X51007846270.txt
X51005442384.txt
X51005568829.txt
X51006619566.txt
X51006619697.txt
X51006557188.txt
X51005757346.txt
X51005763940.txt
X51006913051.txt
X00016469672.txt
X51006557168.txt
X51008145450.txt
X51007262315.txt
X51006679216.txt
X51005705760.txt
X51005442361.txt
X51006311780.txt
X00016469619.txt
X51005441408.txt
X51006502529.txt
X51006857265.txt
X51006619783.txt
X51008164525.txt
X51008164992.txt
X51005444033.txt
X51005757273.txt
X51006619342.txt
X51006008091.txt
X51006329399.txt
X51006913068.txt
X51005453801.txt
X51007339643.txt
X51005719893.txt
X51005663280.txt
X51005711403.txt
X51008099083.txt
X51006414519.txt
X51006502533.txt
X51007846357.txt
X51005433541.txt
X51006335314.txt
X51008142034.txt
X51005717526.txt
X51007103692.txt
X51005442386.txt
X51005568884.txt
X51007339107.txt
X51007339113.txt
X51006557210.txt
X51005605286.txt
X51007339117.txt
X51006388065.txt
X51005711451.txt
X51009453801.t

Revisar spacy test

In [ ]:
resul= []
ruta ='/tmp/SROIE2019v02/SROIE2019/test/'
#archivo = 'X51006619842.txt'
#resul=entidades_total_prueba (ruta,archivo)
for archivo in l_box_test_archivo:
  resul=entidades_total_prueba (ruta,archivo)

X51005587261.txt
X51008099047.txt
X51006828201.txt
X51005724611.txt
X51005337877.txt
X51006334927.txt
X51005361912.txt
X51005568894.txt
X51006387660.txt
X51007339109.txt
X51007339134.txt
X51005677333.txt
X51005433556.txt
X51005711444.txt
X51007391390.txt
X51008030561.txt
X51006387950.txt
X51006556809.txt
X51008099086.txt
X51006389898.txt
X51005663274.txt
X51005675914.txt
X51005444044.txt
X51005711449.txt
X51006619569.txt
X51006557187.txt
X51007103649.txt
X51005301666.txt
X51005806696.txt
X51005757308.txt
X51007231346.txt
X51005230657.txt
X51005724622.txt
X51006387737.txt
X51006332641.txt
X51006334742.txt
X51005724626.txt
X51006620182.txt
X51005663309.txt
X51005741929.txt
X51006619509.txt
X51005605332.txt
X51005763964.txt
X51006466070.txt
X51007339125.txt
X51006857132.txt
X51007846266.txt
X51008042780.txt
X51006349085.txt
X51005746210.txt
X51006414433.txt
X51005719857.txt
X51006557507.txt
X51007231343.txt
X51005568885.txt
X51008042781.txt
X51007846371.txt
X51005444041.txt
X51007846355.t

In [ ]:
textos_nerd = joblib.Parallel(n_jobs=-1)(joblib.delayed(entidades_total_prueba)('/tmp/SROIE2019v02/SROIE2019/train/',archivo) for archivo in tqdm(l_box_archivo))

 34%|███▍      | 214/626 [01:11<01:49,  3.77it/s]/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 626/626 [03:16<00:00,  3.19it/s]


In [ ]:
textos_nerd_test = joblib.Parallel(n_jobs=-1)(joblib.delayed(entidades_total_prueba)('/tmp/SROIE2019v02/SROIE2019/test/',archivo) for archivo in tqdm(l_box_test_archivo))

100%|██████████| 347/347 [01:41<00:00,  3.42it/s]


##Parametros

In [ ]:
labels= ['company', 'date', 'address', 'total','O']

for label in labels:
    ner.add_label(label)

optimizer = nlp.begin_training()

epochs = 94
loss = 0
n_epoch = 500

In [ ]:
spans_train = textos_nerd
spans_test = textos_nerd_test

In [ ]:
print(spans_train)

[("PASARAYA BORONG PINTAR SDN BHD BR NO.: (124525-H) NO 19-G& 19-1& 19-2 JALAN TASIK UTAMA 4, MEDAN NIAGA TASIK DAMAI 016-5498845. GST NO.: 04/03/2018 15:41:52 TAX INVOICE TRN: CR0007636 COUNTER 4 CASHIER: 11 QTY UOM UNIT AMT EXC. AMT INC. GST PRICE TAX TAX CODE POKKA COFFEE VANILLA MILK COFFEE 1 WALK 3.20 3.20 3.20 ZRL *TOTAL QTY: 1.00 3.20 TOTAL INCLUDES GST 0% 3.20 CUSTOMER'S PAYMENT CASH 5.20 CHANGE 2.00 GST SUMMARY AMOUNT (RM) TAX (RM) ZRL = 0% 3.20 0.00 *GOODS SOLD ARE NOT RETURNABLE THANK YOU FOR SHOPPING AT PASARAYA BORONG PINTAR SDN BHD", {'entities': [(0, 8, 'O'), (9, 15, 'O'), (16, 22, 'O'), (23, 26, 'O'), (27, 30, 'O'), (31, 33, 'O'), (34, 38, 'O'), (39, 49, 'O'), (50, 114, 'address'), (115, 127, 'O'), (128, 131, 'O'), (132, 136, 'O'), (137, 147, 'date'), (148, 156, 'O'), (157, 160, 'O'), (161, 168, 'O'), (169, 173, 'O'), (174, 183, 'O'), (184, 191, 'O'), (192, 193, 'O'), (194, 202, 'O'), (203, 205, 'O'), (206, 209, 'O'), (210, 213, 'O'), (214, 218, 'O'), (219, 222, 'O'), (

### Entrenamiento

In [ ]:
def metrics_ner(test, nlp):
    num = 0; matches = 0; Nomatches = 0
    metrics = []
    for sent, lab, arch in test:
        doc = nlp(sent)
        preds = list(map(lambda ent: ent.label_, doc.ents))
        y_true = list(map(lambda case: case[2], lab["entities"]))
        num += len(y_true)
        matches += sum(pred == true for pred, true in zip(preds, y_true))
        Nomatches += sum(pred != true for pred, true in zip(preds, y_true))
    TruePositive =  matches
    FalseNegatives = Nomatches

    accuracy = TruePositive / num
    precision = 0
    recall = 0
    f1Score = 0
    if TruePositive + FalseNegatives > 0:
      precision = TruePositive / (TruePositive + FalseNegatives)
      recall = TruePositive / (TruePositive + FalseNegatives)
      f1Score = 2 * precision * recall / (precision + recall)
    metrics.append(accuracy)
    metrics.append(precision)
    metrics.append(recall)
    metrics.append(f1Score)
    return metrics

def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annotations in examples:
        pred = ner_model(input_)
        #print('Predicion: ',pred,'Anotacion: ',annotations)
        temp = Example.from_dict(pred, dict.fromkeys(annotations))
        example.append(temp)
    scores = scorer.score(example)
    return scores

for epoch in range(epochs):
    losses = {}
    random.shuffle(spans_train)
    batches = minibatch(spans_train[:n_epoch], size=compounding(4, 32, 1.1))
    for batch in batches:
        examples = []
        for sentence in batch:
            example = Example.from_dict(
                nlp.make_doc(sentence[0]), sentence[1]
                )
            examples.append(example)
        nlp.update(
            examples,
            sgd=optimizer,
            drop=0.5,
            losses=losses,
            )
    mner = metrics_ner(spans_test, nlp)
    print(f"Epoch {epoch + 1:<5}/{epochs:<5}loss:{losses['ner'] - loss:.4f} acc:{mner[0]:.4f} pre:{mner[1]:.4f} recall:{mner[2]:.4f} F1:{mner[3]:.4f}")
    loss = losses["ner"]

Epoch 1    /94   loss:47442.6702 acc:0.9604 pre:0.9604 recall:0.9604 F1:0.9604
Epoch 2    /94   loss:-13283.3042 acc:0.8810 pre:0.9579 recall:0.9579 F1:0.9579
Epoch 3    /94   loss:-14717.3118 acc:0.9320 pre:0.9592 recall:0.9592 F1:0.9592
Epoch 4    /94   loss:-3351.7790 acc:0.9359 pre:0.9543 recall:0.9543 F1:0.9543
Epoch 5    /94   loss:-2223.2761 acc:0.9066 pre:0.9493 recall:0.9493 F1:0.9493
Epoch 6    /94   loss:-1022.3054 acc:0.9165 pre:0.9551 recall:0.9551 F1:0.9551
Epoch 7    /94   loss:-443.7890 acc:0.9362 pre:0.9613 recall:0.9613 F1:0.9613
Epoch 8    /94   loss:-915.4864 acc:0.9376 pre:0.9579 recall:0.9579 F1:0.9579
Epoch 9    /94   loss:-784.8007 acc:0.9153 pre:0.9572 recall:0.9572 F1:0.9572
Epoch 10   /94   loss:147.0777 acc:0.9204 pre:0.9587 recall:0.9587 F1:0.9587
Epoch 11   /94   loss:-929.1877 acc:0.9259 pre:0.9600 recall:0.9600 F1:0.9600
Epoch 12   /94   loss:-543.7379 acc:0.9261 pre:0.9631 recall:0.9631 F1:0.9631
Epoch 13   /94   loss:-680.7139 acc:0.9493 pre:0.9655 rec

###Valdiación NER

In [ ]:
texto='3-1708032 POPULAR BOOL CO. (M) SDN BHD (CO. NO. 113821-W) (GST REG NO. 001492992000) NO 8, JALAN 7/1188, BESA TAN RAZAK 56000 KUALA LUMPUR, MALAYSIA KSL CITY TEL: 017-7765076 / 7765987 06/02/18 18:47 SLIP NO.: 0010291725 CHEAH PEI NI TRANS: 246761 DESCRIPTION AMOUNT CORR. PEN ZLI-W 7.65 T PB F/ RING FILE W/CLI 6.99 T TOTAL RM INCL. OF GST 14.64 ROUNDING ADJ 0.01 TOTAL RM 14.65 CASH -50.00 CHANGE 35.35 ITEM COUNT GST SUMMARY AMOUNT (RM) TAX (RM) 2 T @ 6% 13.81 0.83 TOTAL SAVINGS 0.00 BE A POPULAR CARD MEMBER AND ENJOY SPECUAL DUSCOUNTS THANK YOU. PLEASE COME AGAUR. WWW.POPULAR.COM.MY BUY CHINESE BOOKS ONLINE WWW.POPULARONLINE.COM.MY'
doc = nlp(texto)
print(doc.text)
displacy.render(doc,style='ent',jupyter=True)

3-1708032 POPULAR BOOL CO. (M) SDN BHD (CO. NO. 113821-W) (GST REG NO. 001492992000) NO 8, JALAN 7/1188, BESA TAN RAZAK 56000 KUALA LUMPUR, MALAYSIA KSL CITY TEL: 017-7765076 / 7765987 06/02/18 18:47 SLIP NO.: 0010291725 CHEAH PEI NI TRANS: 246761 DESCRIPTION AMOUNT CORR. PEN ZLI-W 7.65 T PB F/ RING FILE W/CLI 6.99 T TOTAL RM INCL. OF GST 14.64 ROUNDING ADJ 0.01 TOTAL RM 14.65 CASH -50.00 CHANGE 35.35 ITEM COUNT GST SUMMARY AMOUNT (RM) TAX (RM) 2 T @ 6% 13.81 0.83 TOTAL SAVINGS 0.00 BE A POPULAR CARD MEMBER AND ENJOY SPECUAL DUSCOUNTS THANK YOU. PLEASE COME AGAUR. WWW.POPULAR.COM.MY BUY CHINESE BOOKS ONLINE WWW.POPULARONLINE.COM.MY


In [ ]:
texto='RESTORAN WAN SHENG 002043319-W NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR GST REG NO: 001335787520 TAX INVOICE INV NO.: 1136284 CASHIER: NICOLE DATE : 11-05-2018 17:12:34 DESCRIPTION QTY U.PRICE TOTAL TAX BUNGA KEKWA 1 X 1.70 1.70 SR TAKE AWAY 1 X 0.20 0.20 SR TOTAL QTY: 2 TOTAL (EXCLUDING GST): 1.79 GST PAYABLE (6%): 0.11 TOTAL (INCLUSIVE OF GST): 1.90 TOTAL : 1.90 CASH : 1.90 GST SUMMARY AMOUNT(RM) TAX(RM) SR (@ 6%) 1.79 0.11'
doc = nlp(texto)
print(doc.text)
displacy.render(doc,style='ent',jupyter=True)

RESTORAN WAN SHENG 002043319-W NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR GST REG NO: 001335787520 TAX INVOICE INV NO.: 1136284 CASHIER: NICOLE DATE : 11-05-2018 17:12:34 DESCRIPTION QTY U.PRICE TOTAL TAX BUNGA KEKWA 1 X 1.70 1.70 SR TAKE AWAY 1 X 0.20 0.20 SR TOTAL QTY: 2 TOTAL (EXCLUDING GST): 1.79 GST PAYABLE (6%): 0.11 TOTAL (INCLUSIVE OF GST): 1.90 TOTAL : 1.90 CASH : 1.90 GST SUMMARY AMOUNT(RM) TAX(RM) SR (@ 6%) 1.79 0.11


In [ ]:
for ent in doc.ents:
  if ent.label_ != 'O':
    print('Entidad: ',ent.label_,' Valor: ',ent.text)

Entidad:  company  Valor:  RESTORAN WAN SHENG
Entidad:  address  Valor:  NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR
Entidad:  total  Valor:  1.90


###Guardar el Modelo

In [ ]:
joblib.dump(nlp, '/content/drive/MyDrive/modelo_ner_jdmc.pkl') # Guardo el modelo.
pnlp = joblib.load('/content/drive/MyDrive/modelo_ner_jdmc.pkl') # Carga del modelo.

In [ ]:
texto='RESTORAN WAN SHENG 002043319-W NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR GST REG NO: 001335787520 TAX INVOICE INV NO.: 1136284 CASHIER: NICOLE DATE : 11-05-2018 17:12:34 DESCRIPTION QTY U.PRICE TOTAL TAX BUNGA KEKWA 1 X 1.70 1.70 SR TAKE AWAY 1 X 0.20 0.20 SR TOTAL QTY: 2 TOTAL (EXCLUDING GST): 1.79 GST PAYABLE (6%): 0.11 TOTAL (INCLUSIVE OF GST): 1.90 TOTAL : 1.90 CASH : 1.90 GST SUMMARY AMOUNT(RM) TAX(RM) SR (@ 6%) 1.79 0.11'
doc = nlp(texto)
print(doc.text)
displacy.render(doc,style='ent',jupyter=True)

RESTORAN WAN SHENG 002043319-W NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR GST REG NO: 001335787520 TAX INVOICE INV NO.: 1136284 CASHIER: NICOLE DATE : 11-05-2018 17:12:34 DESCRIPTION QTY U.PRICE TOTAL TAX BUNGA KEKWA 1 X 1.70 1.70 SR TAKE AWAY 1 X 0.20 0.20 SR TOTAL QTY: 2 TOTAL (EXCLUDING GST): 1.79 GST PAYABLE (6%): 0.11 TOTAL (INCLUSIVE OF GST): 1.90 TOTAL : 1.90 CASH : 1.90 GST SUMMARY AMOUNT(RM) TAX(RM) SR (@ 6%) 1.79 0.11
